# Path Optimization (KOMO)



## Demo of features in Inverse Kinematics

Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [1]:
import sys
sys.path.append('../rai/rai/ry')
import numpy as np
import libry as ry

C = ry.Config()
D = C.view()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../test/kitchen.g')

Let's add some objects

In [2]:
C.addObject(name='item1', parent='sink1', shape=ry.ST.ssBox, pos=[-.1, -.1, .52], size=[.1, .1, .25, .02], color=[1., 0., 0.])
C.addObject('item2', 'sink1', ry.ST.ssBox, [.1, .1, .25, .02], [1., 1., 0.], [.1, .1, .52])
C.addObject('tray', 'stove1', ry.ST.ssBox, [.2, .2, .05, .02], [0., 1., 0.], [.0, .0, .42])

compute a collision free path to touch object item1 with pr2L:

In [3]:
X0 = C.getFrameState()

obj1 = "item1";
arm = "pr2R";

komo = C.komo_path(1.,20, 10., True);

komo.addObjective([], ry.OT.sos, ry.FS.transAccelerations, [], [1.]);
komo.addObjective([], ry.OT.eq, ry.FS.accumulatedCollisions);
komo.addObjective([], ry.OT.ineq, ry.FS.jointLimits);
komo.addObjective([1.], ry.OT.eq, ry.FS.distance, [arm, obj1]);
komo.addObjective([.9,1.], ry.OT.sos, ry.FS.positionDiff, ["endeffWorkspace", obj1], [1e0]);
komo.addObjective(time=[1.], type=ry.OT.eq, feature=ry.FS.qItself, order=1);

komo.optimize(True)
komo.getReport()

[{'confs': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'description': 'QuaternionNorms',
  'eq_sumOfAbs': 0.0,
  'type': 'eq'},
 {'confs': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'feature': 'Transition',
  'sos_value': 3.1548225930515295,
  'type': 'sos'},
 {'confs': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'eq_sumOfAbs': 0.00022084378362776172,
  'feature': 'ProxyCost',
  'type': 'eq'},
 {'confs': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'feature': 'LimitsConstraint',
  'inEq_sumOfPos': 0.0,
  'type': 'ineq'},
 {'confs': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  'eq_sumOfAbs': 0.0010835191783960564,
  'feature': 'dist',
  'o1': 'pr2R',
  'o2': 'item1',
  'type': 'eq'},
 {'confs': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
  'feature': 'posDiff',
  'o1': 'endeffWorkspace',
  'o2': 'item1',
  'sos_value': 0.5625589126906487,
  'type': 'sos'},
 {'confs': [0, 

In [4]:
V = komo.display()

In [5]:
#komo.displayTrajectory()
V = komo.view()

In [6]:
V=0

Best way to read out: grab a configuration into K and analyze it here:

In [2]:
C.setFrameState( komo.getConfiguration(19) )
C.getJointState()

NameError: name 'C' is not defined